In [158]:
import numpy as np
import pandas as pd
import logging
from graphviz import Digraph

In [6]:
logging.basicConfig(filename='./log.txt',level=logging.DEBUG)


In [61]:
def relu(x):
    return x


In [154]:
class layer:

    def __init__(self, num_nodes, next_num_nodes = 1, initial_weight = None, actn_fxn = relu):

        self.num_nodes = num_nodes
        self.actn_fxn  = actn_fxn
        self.next_num_nodes = next_num_nodes

        self.weight_matrix = None
        
        if initial_weight is None:
            logging.debug('Using default values for the weight matrix')
            self.weight_matrix = np.zeros([self.num_nodes, self.next_num_nodes],dtype=float)
        
        elif isinstance(initial_weight, (int,float)):
            logging.debug("Using the weight value for all weights in the layer")
            self.weight_matrix = (np.zeros(shape = (self.num_nodes, self.next_num_nodes),dtype=float))
            self.weight_matrix.fill(float(initial_weight))
            
        elif initial_weight.shape == (self.num_nodes, self.next_num_nodes):
            logging.debug("Using user provided values")
            self.weight_matrix = np.array(initial_weight, dtype = float)
            
        
            
        else:
            logging.warning('Weight matrix provided is of incompatible size. Using default values')
            self.weight_matrix = np.zeros(shape = (self.num_nodes,self.next_num_nodes),dtype = float)
        
        self.input_vector = None
        
    def send(self, input_vector):
        
        #flattened to keep things simple
        input_vector = np.array(input_vector).flatten()
        
        if input_vector.shape != (self.num_nodes,):
            logging.critical("Input vector not of desired size. Cannot continue!")
            return None
        
        self.input_vector = np.asarray([ self.actn_fxn(i) for i in input_vector ])
            
    
    def generate(self):
        
        if self.input_vector is None:
            logging.critical("Input not yet provided. Cannot continue!")
            return None
        
        #the input is required to be in a row only
        temp_input_matrix = np.reshape(self.input_vector, newshape=(1,self.num_nodes))
        
        #input is consumed
        self.input_vector = None
        
        output_matrix = np.dot(temp_input_matrix,self.weight_matrix)
        
        return output_matrix.flatten()
        
        
    def desc(self):
        print("Number of Nodes: {}\n Number of Nodes in next layer: {}\nWeights:\n {}\n"
               .format(self.num_nodes, self.next_num_nodes,self.weight_matrix))
        
        
        

In [187]:
class n_ff_network:
    
    def __init__(self, ff_layers, initial_weights = None, actn_fxn = relu):
        
        self.ff_layers = list(np.array(ff_layers).flatten())
        self.num_layers = len(self.ff_layers)
        
        self.n_layers = None
        
        if isinstance(initial_weights,(type(None),int,float)):
            self.n_layers = [ layer(num_nodes = self.ff_layers[i], next_num_nodes= self.ff_layers[i+1], 
                                    actn_fxn = actn_fxn, initial_weight= initial_weights)
                               for i in range(self.num_layers - 1)]
        
        elif initial_weight.shape[0] == self.num_layers - 1:
            self.n_layers = [ layer(num_nodes = self.ff_layers[i], next_num_nodes= self.ff_layers[i+1], 
                                    actn_fxn = actn_fxn, initial_weight= initial_weights[i])
                               for i in range(self.num_layers - 1)]
        
        self.n_layers.append(layer(num_nodes = self.ff_layers[self.num_layers-1],initial_weight=1, 
                                       next_num_nodes=self.ff_layers[self.num_layers-1], actn_fxn = actn_fxn))
            
    def desc(self):
        for i in range(self.num_layers):
            print("Layer {}\n".format(i+1))
            self.n_layers[i].desc()
            
    def show_network(self):
        dot = Digraph(comment='Neural Network')
        
        start = 0
        
        for i in range(self.num_layers):
            for j in range(self.n_layers[i].num_nodes):
                dot.node(name = str(start),label = 'x'+str(i)+str(j))
                start +=1
        
        start = 0
        for i in range(self.num_layers-1):
            curr_layer = self.n_layers[i]
            next_start = start + curr_layer.num_nodes
            
            tail = start
            head = next_start
            for j in range(curr_layer.num_nodes):
                head = next_start
                for k in range(curr_layer.next_num_nodes):
                    dot.edge(tail_name = str(tail),head_name = str(head), label = str(curr_layer.weight_matrix[j][k]))
                    head += 1
                tail += 1
            
            start = next_start
        
        dot.render('./temp.gv', view = True)
                

In [190]:
a = n_ff_network(ff_layers=[3,2,3],initial_weights=3)
a.show_network()